In [1]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import scipy.io.wavfile

import keras
from keras.utils import np_utils

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class VCBSequence(keras.utils.Sequence):
    
    def __init__(self, partition, crop_length=2*16000, batch_size=512,
                 shuffle=True, random_seed=42,
                 sr=16000,
                 vcb_root="/home/ubuntu/data/voxceleb1_wav",
                 split_txt="/home/ubuntu/data/id-split.txt"):
        """
        partition: one of "train", "val", "test"
        crop_length: length in samples
        """
        partition_map = {
            'train': 1,
            'val': 2,
            'test': 3
        }
        self.partition = partition_map[partition]
        
        self.sr = sr
        self.crop_length = crop_length
        self.batch_size = batch_size
        self.vcb_root = vcb_root
        self.split_txt = split_txt
        
        with open(self.split_txt) as f:
            lines = [line.rstrip() for line in f.readlines()]
        if shuffle:
            np.random.seed(random_seed)
            np.random.shuffle(lines)
        
        self.paths = [line.split(" ") for line in lines]
        for i in range(len(self.paths)):
            self.paths[i][0] = int(self.paths[i][0])
        
        self.all_names = [p[1].split("/")[0] for p in self.paths]
        self.encoder = LabelEncoder()
        self.encoder.fit(self.all_names)
        self.classes = np_utils.to_categorical(self.encoder.transform(self.all_names))
        keep = list(filter(lambda x: x[1][0] == self.partition, enumerate(self.paths)))
        keep = [x[0] for x in keep]
        self.classes = self.classes[keep]
            
        self.paths = list(filter(lambda x: x[0] == self.partition, self.paths))
        self.paths = [p[1] for p in self.paths]
        
    def __len__(self):
        ret = len(self.paths) // self.batch_size
        if len(self.paths) % len(self.paths) != 0:
            ret += 1
        return ret

    def get_random_cropped_wavfile(self, path):
        rate, data = scipy.io.wavfile.read(self.vcb_root + "/" + path)
        assert rate == self.sr
        length = len(data)
        crop_start = np.random.randint(length - self.crop_length)
        crop = data[crop_start : crop_start + self.crop_length]
        return crop
    
    def __getitem__(self, idx):
        batch_paths = self.paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        x = np.zeros((len(batch_paths), self.sr * self.crop_length))
        for i, p in enumerate(batch_paths):
            x[i] = self.get_random_cropped_wavfile(p)
        
        y = self.classes[idx * self.batch_size : min((idx + 1) * self.batch_size,
                                                     self.classes.shape[0])]
        
        return x, y

In [4]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense

import kapre
from kapre.time_frequency import Melspectrogram
from utils.normalization2d import Normalization2D


HIDDEN_DIM = 256

model = Sequential()

SR = 16000 # sampling rate
CROP_LENGTH = 128 * 256
BATCH_SIZE = 512
#train_gen = VCBSequence('train', sr=SR, crop_length=CROP_LENGTH, batch_size=BATCH_SIZE)
#val_data = VCBSequence('val', sr=SR, crop_length=CROP_LENGTH, batch_size=10000)[0]
#test_gen = VCBSequence('test', sr=SR, crop_length=CROP_LENGTH, batch_size=BATCH_SIZE)
model.add(keras.layers.Reshape((1, -1), input_shape=(None,)))

N_MELS = 512
N_FFT = 1024
N_HOP = 128
model.add(Melspectrogram(sr=SR, n_mels=N_MELS, 
      n_dft=N_FFT, n_hop=N_HOP, 
      return_decibel_melgram=True, power_melgram=2.0,
      trainable_kernel=False, name='melgram', input_shape=(1, None)))
model.add(Normalization2D(int_axis=2))
model.add(keras.layers.Reshape((N_MELS, -1)))
model.add(keras.layers.Permute((2, 1)))
model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(keras.layers.GlobalAveragePooling1D())

NUM_PEOPLE = len(VCBSequence('test').encoder.classes_)
model.add(Dense(NUM_PEOPLE, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 1, None)           0         
_________________________________________________________________
melgram (Melspectrogram)     (None, 512, None, 1)      1313280   
_________________________________________________________________
normalization2d_1 (Normaliza (None, 512, None, 1)      0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 512, None)         0         
_________________________________________________________________
permute_1 (Permute)          (None, None, 512)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         787456    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 256)               0         
__________

In [ ]:
print(val_data[0].shape, val_data[1].shape)

In [ ]:
model = keras.models.load_model("/home/ubuntu/models/save/2second-crop-05-0.28.hdf5",
                                custom_objects={
                                    "Melspectrogram": Melspectrogram,
                                    "Normalization2D": kapre.utils.Normalization2D
                                })

In [ ]:
LEARNING_RATE = 1e-3
rmsprop = keras.optimizers.Adam(lr=LEARNING_RATE)

def top_k(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=5) # change k here
model.compile(optimizer=rmsprop,
              loss='categorical_crossentropy', # loss function to be optimized
              metrics=['accuracy', top_k]) # doesn't affect optimization

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from utils.custom_callbacks import TensorBoard

checkpoint_filepath="/home/ubuntu/models/2second-crop-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

tensorboard = TensorBoard(log_dir="/home/ubuntu/tflogs/cur",
                          write_batch_performance=True,
                          write_graph=False,
                          batch_size=BATCH_SIZE)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=1e-5)

In [ ]:
model.fit_generator(train_gen, validation_data=val_data, epochs=30,
                    callbacks=[checkpoint, tensorboard, reduce_lr],
                    workers=5)

In [ ]:
model.evaluate_generator(train_gen)

In [ ]:
model.evaluate_generator(val_gen)

In [ ]:
model.evaluate_generator(test_gen)